In [ ]:
import os
import sys
import time
import random
import itertools
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from functools import partial
from itertools import cycle, islice
from multiprocessing import Pool
import multiprocessing
import warnings
from sklearn import datasets, metrics, linear_model, svm
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.datasets import (make_moons, make_circles, make_classification, 
                             make_blobs, make_checkerboard)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              ExtraTreesClassifier, GradientBoostingClassifier, 
                              BaggingClassifier)
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

from imblearn.over_sampling import SMOTE
import xgboost as xgb
from scipy import interp


savepath='E:/Workspace/Data/20240625_AKI_processed/'

df=pd.read_csv('20240625_AKI_data.csv',encoding='cp949')
len(df)

In [ ]:
def find_all_files_in_folders(top_folders):
    all_files = []

    def find_files(root_folder, current_path=""):
        nonlocal all_files

        for item in os.listdir(os.path.join(root_folder, current_path)):
            item_path = f"{current_path}/{item}" if current_path else item
            full_item_path = os.path.join(root_folder, item_path)

            if os.path.isfile(full_item_path):
                all_files.append(full_item_path)
            else:
                find_files(root_folder, item_path)

    for top_folder in top_folders:
        find_files(top_folder)

    return all_files

top_folders = ["E:/Workspace/Data/Postopdelirium/20240311_postopdelirium_csv", "E:/Workspace/Data/Postopdelirium/20240311_postopdelirium_val_csv", "E:/Workspace/Data/20240311_AKI_csv","E:/Workspace/Data/AKI_val_csv1","E:/Workspace/Data/AKI_val_csv2"]
all_files = find_all_files_in_folders(top_folders)
len(all_files)

In [ ]:
def findfile(csvfilename):
    for i in range(len(all_files)):
        if csvfilename in all_files[i]:
            returnfilename=all_files[i]
            break
    return returnfilename     


def get_anes_start_time(i):
    year=df['Anes_START'][i][:].split('-')[0]
    month=df['Anes_START'][i][:].split('-')[1]
    day=df['Anes_START'][i][:].split('-')[2].split(' ')[0]
    hour=df['Anes_START'][i][:].split(':')[0].split(' ')[1]
    minute=df['Anes_START'][i][:].split(':')[1]
    returntime=datetime.datetime(int(year),int(month),int(day),int(hour),int(minute))
    return returntime

def get_anes_end_time(i):
    year=df['Anes_END'][i][:].split('-')[0]
    month=df['Anes_END'][i][:].split('-')[1]
    day=df['Anes_END'][i][:].split('-')[2].split(' ')[0]
    hour=df['Anes_END'][i][:].split(':')[0].split(' ')[1]
    minute=df['Anes_END'][i][:].split(':')[1]
    returntime=datetime.datetime(int(year),int(month),int(day),int(hour),int(minute))
    return returntime

def get_CPB_on_time(i):
    year=df['CPB_on'][i][:].split('-')[0]
    month=df['CPB_on'][i][:].split('-')[1]
    day=df['CPB_on'][i][:].split('-')[2].split(' ')[0]
    hour=df['CPB_on'][i][:].split(':')[0].split(' ')[1]
    minute=df['CPB_on'][i][:].split(':')[1]
    returntime=datetime.datetime(int(year),int(month),int(day),int(hour),int(minute))
    return returntime

def get_CPB_off_time(i):
    year=df['CPB_off'][i][:].split('-')[0]
    month=df['CPB_off'][i][:].split('-')[1]
    day=df['CPB_off'][i][:].split('-')[2].split(' ')[0]
    hour=df['CPB_off'][i][:].split(':')[0].split(' ')[1]
    minute=df['CPB_off'][i][:].split(':')[1]
    returntime=datetime.datetime(int(year),int(month),int(day),int(hour),int(minute))
    return returntime

def get_TCA_start_time(i):
    year=df['TCA_start'][i][:].split('-')[0]
    month=df['TCA_start'][i][:].split('-')[1]
    day=df['TCA_start'][i][:].split('-')[2].split(' ')[0]
    hour=df['TCA_start'][i][:].split(':')[0].split(' ')[1]
    minute=df['TCA_start'][i][:].split(':')[1]
    returntime=datetime.datetime(int(year),int(month),int(day),int(hour),int(minute))
    return returntime

def get_TCA_end_time(i):
    year=df['TCA_end'][i][:].split('-')[0]
    month=df['TCA_end'][i][:].split('-')[1]
    day=df['TCA_end'][i][:].split('-')[2].split(' ')[0]
    hour=df['TCA_end'][i][:].split(':')[0].split(' ')[1]
    minute=df['TCA_end'][i][:].split(':')[1]
    returntime=datetime.datetime(int(year),int(month),int(day),int(hour),int(minute))
    return returntime

def get_op_start_time(i):
    year=df['Op_START'][i][:].split('-')[0]
    month=df['Op_START'][i][:].split('-')[1]
    day=df['Op_START'][i][:].split('-')[2].split(' ')[0]
    hour=df['Op_START'][i][:].split(':')[0].split(' ')[1]
    minute=df['Op_START'][i][:].split(':')[1]
    returntime=datetime.datetime(int(year),int(month),int(day),int(hour),int(minute))
    return returntime

def get_op_end_time(i):
    year=df['Op_END'][i][:].split('-')[0]
    month=df['Op_END'][i][:].split('-')[1]
    day=df['Op_END'][i][:].split('-')[2].split(' ')[0]
    hour=df['Op_END'][i][:].split(':')[0].split(' ')[1]
    minute=df['Op_END'][i][:].split(':')[1]
    returntime=datetime.datetime(int(year),int(month),int(day),int(hour),int(minute))
    return returntime


def find_first_non_null(row):
    # 'Time' 열을 제외합니다.
    non_time_row = row.drop('Time')
    # 첫 번째 유효한 값의 인덱스를 찾습니다.
    first_valid_index = non_time_row.first_valid_index()
    # 해당 인덱스의 값을 반환합니다.
    return non_time_row[first_valid_index] if first_valid_index else np.nan

In [ ]:
for i in range(len(df)):

    pid=df['NO'][i]
    rSO2_total=pd.DataFrame(columns=['Time','rSO2_L','rSO2_R'])
    cnt=0
    for filefile in ['파일이름_01','파일이름_02','파일이름_03','파일이름_04','파일이름_05','파일이름_06','파일이름_07','파일이름_08']:      
        try:
            dat=pd.read_csv(findfile(df[filefile][i]))
            cnt+=1
            try:
                rSO2_1_columns=['Time']
                rSO2_1_columns2=[col for col in dat.columns if 'SO2_1' in col]
                rSO2_1_columns3=[col for col in dat.columns if 'SCO2_L' in col]
                rSO2_1_columns4=[col for col in dat.columns if 'SCO2_S1' in col]
                rSO2_1_columns=rSO2_1_columns+rSO2_1_columns2+rSO2_1_columns3+rSO2_1_columns4
                rSO2_1=dat[rSO2_1_columns]
                
                rSO2_2_columns=['Time']
                rSO2_2_columns2=[col for col in dat.columns if 'SO2_2' in col]
                rSO2_2_columns3=[col for col in dat.columns if 'SCO2_R' in col]
                rSO2_2_columns4=[col for col in dat.columns if 'SCO2_S2' in col]
                rSO2_2_columns=rSO2_2_columns+rSO2_2_columns2+rSO2_2_columns3+rSO2_2_columns4
                rSO2_2=dat[rSO2_2_columns]
                
                
                rSO2_1=rSO2_1.dropna(axis=0,subset=rSO2_1_columns[1:],how='all')
                rSO2_1.reset_index(inplace=True,drop=True)
                rSO2_2=rSO2_2.dropna(axis=0,subset=rSO2_2_columns[1:],how='all')
                rSO2_2.reset_index(inplace=True,drop=True)
                
                rSO2_1['rSO2_L'] = rSO2_1.apply(find_first_non_null, axis=1)
                rSO2_2['rSO2_R'] = rSO2_2.apply(find_first_non_null, axis=1)
                
                
                rSO2_1_temp=rSO2_1[['Time','rSO2_L']]
                rSO2_2_temp=rSO2_2[['Time','rSO2_R']]
                
                rSO2=pd.merge(rSO2_1_temp,rSO2_2_temp,how='outer',on='Time')
                rSO2=rSO2.dropna(axis=0)
                rSO2.reset_index(inplace=True,drop=True)

                rSO2_total=pd.concat([rSO2_total,rSO2])
            except:
                print('==========',i,'wrong')

        except:
            pass


    rSO2_total=rSO2_total.dropna(axis=0)
    if len(rSO2_total)>=0:
        rSO2_total=rSO2_total[rSO2_total['rSO2_L']!=0]
        rSO2_total=rSO2_total[rSO2_total['rSO2_R']!=0]
        rSO2_total.sort_values(by=['Time'],axis=0,inplace=True)
        rSO2_total.reset_index(inplace=True,drop=True)
        rSO2_total['rSO2_mean']=(rSO2_total['rSO2_L']+rSO2_total['rSO2_R'])/2



        rSO2_total['year']=rSO2_total['Time'].str[:4]
        rSO2_total['month']=rSO2_total['Time'].str[5:7]
        rSO2_total['day']=rSO2_total['Time'].str[8:10]
        rSO2_total['hour']=rSO2_total['Time'].str[11:13]
        rSO2_total['minute']=rSO2_total['Time'].str[14:16]
        rSO2_total['second']=rSO2_total['Time'].str[17:19]
        rSO2_total['year']=pd.to_numeric(rSO2_total['year'])
        rSO2_total['month']=pd.to_numeric(rSO2_total['month'])
        rSO2_total['day']=pd.to_numeric(rSO2_total['day'])
        rSO2_total['hour']=pd.to_numeric(rSO2_total['hour'])
        rSO2_total['minute']=pd.to_numeric(rSO2_total['minute'])
        rSO2_total['second']=pd.to_numeric(rSO2_total['second'])
        rSO2_total['second2']=(rSO2_total['second']//5)*5

        rSO2_total['timetime']=rSO2_total['Time'].str[:17]+('0'+rSO2_total['second2'].astype(str)).str[-2:]
        rSO2_total['datetime']=pd.to_datetime(rSO2_total['timetime'],format="%Y-%m-%d %H:%M:%S")

        rSO2_total2=rSO2_total[['rSO2_mean','datetime']]

        rSO2_total2.drop_duplicates(inplace=True)
        rSO2_total2.reset_index(inplace=True,drop=True)
        
        if len(rSO2_total2)>0:
            rSO2_total2.to_csv(savepath+'rSO2/'+str(pid)+'.csv',index=False)
            print(i,pid)
        else:
            print('========error2,pid',pid,'length 0')
    else:
        print('========error1,pid',pid,'length 0')

In [ ]:
avail=[]

for i in range(len(df)):
    try:
        pid=df['NO'][i]
        anes_end_time=get_anes_end_time(i)
        anes_start_time=get_anes_start_time(i)

        total_anes_time=(anes_end_time-anes_start_time).seconds
        rSO2_df=pd.read_csv(savepath+'rSO2/'+str(pid)+'.csv')
        rSO2_df['datetime']=pd.to_datetime(rSO2_df['datetime'],format="%Y-%m-%d %H:%M:%S")
        count=0
        for j in range(len(rSO2_df)):

            timestamp=rSO2_df['datetime'][j]
            if timestamp>=anes_start_time and timestamp<=anes_end_time:
                count+=1
        avail.append(count/(total_anes_time//5))
        print(i)
    except:
        print('==========pid',pid,'error')  
        avail.append(np.nan)

df['rSO2_avail']=avail

In [ ]:
plt.hist(df['rSO2_avail'])

In [ ]:
baseline_rSO2_list=[]

for i in range(len(df)):
    if True:
        try:
            pid=df['NO'][i]
            rSO2_df=pd.read_csv(savepath+"rSO2/"+str(pid)+'.csv')
            rSO2_df['datetime']=pd.to_datetime(rSO2_df['datetime'],format="%Y-%m-%d %H:%M:%S")
            try:

                anes_start_time=get_anes_start_time(i)

                op_start_time=get_op_start_time(i)

                first_rSO2=0
                first_rSO2_time=0

                for j in range(len(rSO2_df)):

                    if rSO2_df['datetime'][j]>=anes_start_time:
                        first_rSO2=rSO2_df['rSO2_mean'][j]
                        first_rSO2_time=rSO2_df['datetime'][j]
                        break

                baselines=[]
                if first_rSO2_time!=0:
                    for j in range(12):
                        thetime=first_rSO2_time+datetime.timedelta(seconds=5*j)
                        try:
                            therSO2=rSO2_df[rSO2_df['datetime']==thetime]['rSO2_mean'].item()
                            baselines.append(therSO2)
                        except:
                            baselines.append(np.nan)
                    baselines=pd.Series(baselines)
                    baselines.interpolate(inplace=True)
                    baselines=list(baselines)
                    baseline_rSO2=np.nanmean(baselines)
                    if np.isnan(baseline_rSO2)==False:
                        baseline_rSO2_list.append(baseline_rSO2)
                    else:
                        baseline_rSO2_list.append(np.nan)
                        print('======nan1======pid',pid)
                else:
                    baseline_rSO2_list.append(np.nan)
                    print('======nan2======pid',pid)


            except:
                print('=======error1 pid',pid)
                baseline_rSO2_list.append(np.nan)
        except:
            print('=======error2 pid',pid)
            baseline_rSO2_list.append(np.nan)
    else:
        print('excluded')
        baseline_rSO2_list.append(np.nan)
df['baseline_rSO2']=baseline_rSO2_list

In [ ]:
plt.hist(df['baseline_rSO2'])

In [ ]:
for i in range(len(df)):
    pid=df['NO'][i]
    
    MAP_total=pd.DataFrame(columns=['Time','MAP'])
    CVP_total=pd.DataFrame(columns=['Time','CVP'])

    for filefile in ['파일이름_01','파일이름_02','파일이름_03','파일이름_04','파일이름_05','파일이름_06','파일이름_07','파일이름_08']:      
        try:
            dat=pd.read_csv(findfile(df[filefile][i]))
            try:
                MAP_columns=['Time']
                MAP_columns2=[col for col in dat.columns if 'ART_MEAN' in col]
                MAP_columns3=[col for col in dat.columns if 'ABP_MEAN' in col]
                MAP_columns=MAP_columns+MAP_columns2+MAP_columns3
                MAP=dat[MAP_columns]
                MAP=MAP.dropna(axis=0,subset=MAP_columns[1:],how='all')
                MAP.reset_index(inplace=True,drop=True)
                
                CVP_columns=['Time']
                CVP_columns2=[col for col in dat.columns if 'CVP_MEAN' in col]
                CVP_columns=CVP_columns+CVP_columns2
                CVP=dat[CVP_columns]
                CVP=CVP.dropna(axis=0,subset=CVP_columns[1:],how='all')
                CVP.reset_index(inplace=True,drop=True)
                
                MAP['MAP_final'] = MAP.apply(find_first_non_null, axis=1)
                CVP['CVP_final'] = CVP.apply(find_first_non_null, axis=1)
                
                MAP_temp=MAP[['Time','MAP_final']]
                CVP_temp=CVP[['Time','CVP_final']]
                
                MAP_temp.columns=['Time','MAP']
                CVP_temp.columns=['Time','CVP']
                
                MAP_total=pd.concat([MAP_total,MAP_temp])
                CVP_total=pd.concat([CVP_total,CVP_temp])

            except:
                print('==========',i,pid,'wrong')

        except:
            pass
    MAP_total.sort_values(by=['Time'],axis=0,inplace=True)
    MAP_total.drop_duplicates(inplace=True,subset=['Time'],keep='first')
    MAP_total.reset_index(inplace=True,drop=True)

    CVP_total.sort_values(by=['Time'],axis=0,inplace=True)
    CVP_total.drop_duplicates(inplace=True,subset=['Time'],keep='first')
    CVP_total.reset_index(inplace=True,drop=True)
    
    MAP_CVP_total=pd.merge(MAP_total,CVP_total,how='outer',on='Time')

    MAP_CVP_total.sort_values(by=['Time'],axis=0,inplace=True)

    MAP_CVP_total.reset_index(inplace=True,drop=True)

    MAP_CVP_total['year']=MAP_CVP_total['Time'].str[:4]
    MAP_CVP_total['month']=MAP_CVP_total['Time'].str[5:7]
    MAP_CVP_total['day']=MAP_CVP_total['Time'].str[8:10]
    MAP_CVP_total['hour']=MAP_CVP_total['Time'].str[11:13]
    MAP_CVP_total['minute']=MAP_CVP_total['Time'].str[14:16]
    MAP_CVP_total['second']=MAP_CVP_total['Time'].str[17:19]
    MAP_CVP_total['year']=pd.to_numeric(MAP_CVP_total['year'])
    MAP_CVP_total['month']=pd.to_numeric(MAP_CVP_total['month'])
    MAP_CVP_total['day']=pd.to_numeric(MAP_CVP_total['day'])
    MAP_CVP_total['hour']=pd.to_numeric(MAP_CVP_total['hour'])
    MAP_CVP_total['minute']=pd.to_numeric(MAP_CVP_total['minute'])
    MAP_CVP_total['second']=pd.to_numeric(MAP_CVP_total['second'])
    MAP_CVP_total['second2']=(MAP_CVP_total['second']//5)*5

    MAP_CVP_total['timetime']=MAP_CVP_total['Time'].str[:17]+('0'+MAP_CVP_total['second2'].astype(str)).str[-2:]
    MAP_CVP_total['datetime']=pd.to_datetime(MAP_CVP_total['timetime'],format="%Y-%m-%d %H:%M:%S")


    MAP_CVP_total2=MAP_CVP_total[['datetime','MAP', 'CVP']]
    
    MAP_CVP_total2_MAP=MAP_CVP_total2[['datetime','MAP']]
    MAP_CVP_total2_MAP.drop_duplicates(inplace=True,subset=['datetime'],keep='first')
    MAP_CVP_total2_MAP.reset_index(inplace=True,drop=True)
    
    MAP_CVP_total2_CVP=MAP_CVP_total2[['datetime','CVP']]
    MAP_CVP_total2_CVP.drop_duplicates(inplace=True,subset=['datetime'],keep='first')
    MAP_CVP_total2_CVP.reset_index(inplace=True,drop=True)
    
    MAP_CVP_total2=pd.merge(MAP_CVP_total2_MAP,MAP_CVP_total2_CVP,how='outer',on='datetime')

    MAP_CVP_total2.drop_duplicates(inplace=True)
    MAP_CVP_total2.reset_index(inplace=True,drop=True)

    MAP_CVP_total2['MAP_drop']=0
    for j in range(len(MAP_CVP_total2)):
        if MAP_CVP_total2['MAP'][j]<=30:
            droptimelist=[]
            for k in range(-6,7):
                droptime=MAP_CVP_total2['datetime'][j]+datetime.timedelta(seconds=5*k)
                droptimelist.append(droptime)
            for k in range(len(droptimelist)):
                try:
                    MAP_CVP_total2.loc[MAP_CVP_total2['datetime'] == droptimelist[k], 'MAP_drop'] = 1
                except:
                    pass

    MAP_CVP_total2['CVP_drop']=0
    for j in range(len(MAP_CVP_total2)):
        if MAP_CVP_total2['CVP'][j]>=100:
            droptimelist=[]
            for k in range(-6,7):
                droptime=MAP_CVP_total2['datetime'][j]+datetime.timedelta(seconds=5*k)
                droptimelist.append(droptime)
            for k in range(len(droptimelist)):
                try:
                    MAP_CVP_total2.loc[MAP_CVP_total2['datetime'] == droptimelist[k], 'CVP_drop'] = 1
                except:
                    pass


    MAP_CVP_total2.to_csv(savepath+"MAP_CVP/"+str(pid)+'.csv')

    if i%100==0:
        print(i)


In [ ]:
MAP_avail=[]
CVP_avail=[]


for i in range(len(df)): 
    pid=df['NO'][i]
    if True: 
        if df['op_type'][i]!=0 : 
            try:
                anes_end_time=get_anes_end_time(i)
                anes_start_time=get_anes_start_time(i)
                CPB_on_time=get_CPB_on_time(i)
                CPB_off_time=get_CPB_off_time(i)

                total_anes_minus_CPB_time=(anes_end_time-anes_start_time).seconds-(CPB_off_time-CPB_on_time).seconds
                try:
                    TCA_start_time=get_TCA_start_time(i)
                    TCA_end_time=get_TCA_end_time(i)
                    total_anes_minus_TCA_time=(anes_end_time-anes_start_time).seconds-(TCA_end_time-TCA_start_time).seconds
                except:
                    total_anes_minus_TCA_time=(anes_end_time-anes_start_time).seconds

                MAP_CVP_df=pd.read_csv(savepath+"MAP_CVP/"+str(pid)+'.csv')
                MAP_CVP_df['datetime']=pd.to_datetime(MAP_CVP_df['datetime'],format="%Y-%m-%d %H:%M:%S")
                MAP_count=0
                CVP_count=0
                for j in range(len(MAP_CVP_df)):

                    timestamp=MAP_CVP_df['datetime'][j]
                    if (timestamp>=anes_start_time and timestamp<=CPB_on_time) or (timestamp>=CPB_off_time and timestamp<=anes_end_time):
                        CVP_count+=1
                    try:
                        TCA_start_time=get_TCA_start_time(i)
                        TCA_end_time=get_TCA_end_time(i)
                        if (timestamp>=anes_start_time and timestamp<=TCA_start_time) or (timestamp>=TCA_end_time and timestamp<=anes_end_time):
                            MAP_count+=1
                    except:
                        if timestamp>=anes_start_time and timestamp<=anes_end_time:
                            MAP_count+=1

                MAP_avail.append(MAP_count/(total_anes_minus_TCA_time//5))          
                CVP_avail.append(CVP_count/(total_anes_minus_CPB_time//5))
                if MAP_count/(total_anes_minus_TCA_time//5)<0:
                    print(i,pid)
                    
                if CVP_count/(total_anes_minus_CPB_time//5)<0:
                    print(i,pid)
                
                
                if CVP_count/(total_anes_minus_CPB_time//5)>1.1:
                    print(i,pid)
                #print(i)
            except:
                print(i,'error')  
                MAP_avail.append(0)
                CVP_avail.append(0)
        else:
            MAP_avail.append(0)
            CVP_avail.append(0)
            print(i,pid,'optype wrong')
    else:
        print(i,'====excluded=====')
        MAP_avail.append(0)
        CVP_avail.append(0)
df['MAP_avail']=MAP_avail
df['CVP_avail']=CVP_avail

In [ ]:
plt.hist(MAP_avail)

In [ ]:
plt.hist(CVP_avail)

In [ ]:
MAP_mean_list=[]
MAP_CV_list=[]
MAP_ARV_list=[]
MAP_duration_65_list=[]
MAP_duration_100_list=[]
MAP_auc_65_list=[]
MAP_auc_100_list=[]
PP_mean_list=[]
PP_duration_60_list=[]
PP_auc_60_list=[]
CVP_mean_list=[]
CVP_duration_12_list=[]
CVP_auc_12_list=[]

NEW_MAP_preCPB_mean_list=[]
NEW_MAP_preCPB_CV_list=[]
NEW_MAP_preCPB_ARV_list=[]
NEW_MAP_preCPB_duration_65_list=[]
NEW_MAP_preCPB_duration_100_list=[]
NEW_MAP_preCPB_auc_65_list=[]
NEW_MAP_preCPB_auc_100_list=[]
NEW_PP_preCPB_mean_list=[]
NEW_PP_preCPB_duration_60_list=[]
NEW_PP_preCPB_auc_60_list=[]
NEW_CVP_preCPB_mean_list=[]
NEW_CVP_preCPB_duration_12_list=[]
NEW_CVP_preCPB_auc_12_list=[]

NEW_MAP_intraCPB_mean_list=[]
NEW_MAP_intraCPB_CV_list=[]
NEW_MAP_intraCPB_ARV_list=[]
NEW_MAP_intraCPB_duration_65_list=[]
NEW_MAP_intraCPB_duration_100_list=[]
NEW_MAP_intraCPB_auc_65_list=[]
NEW_MAP_intraCPB_auc_100_list=[]

NEW_MAP_postCPB_mean_list=[]
NEW_MAP_postCPB_CV_list=[]
NEW_MAP_postCPB_ARV_list=[]
NEW_MAP_postCPB_duration_65_list=[]
NEW_MAP_postCPB_duration_100_list=[]
NEW_MAP_postCPB_auc_65_list=[]
NEW_MAP_postCPB_auc_100_list=[]
NEW_PP_postCPB_mean_list=[]
NEW_PP_postCPB_duration_60_list=[]
NEW_PP_postCPB_auc_60_list=[]
NEW_CVP_postCPB_mean_list=[]
NEW_CVP_postCPB_duration_12_list=[]
NEW_CVP_postCPB_auc_12_list=[]

NEW_PP_CPBMAPincluded_mean_list=[]
NEW_PP_CPBMAPincluded_duration_60_list=[]
NEW_PP_CPBMAPincluded_auc_60_list=[]



for i in range(len(df)):

    pid=df['NO'][i]
    try:
        if True: 
            MAP_CVP_df=pd.read_csv(savepath+"MAP_CVP/"+str(pid)+'.csv')
            MAP_CVP_df['datetime']=pd.to_datetime(MAP_CVP_df['datetime'],format="%Y-%m-%d %H:%M:%S")

            MAP_mean=np.nan
            MAP_CV=np.nan
            MAP_ARV=np.nan
            MAP_duration_65=np.nan
            MAP_duration_100=np.nan
            MAP_auc_65=np.nan
            MAP_auc_100=np.nan
            PP_mean=np.nan
            PP_duration_60=np.nan
            PP_auc_60=np.nan
            CVP_mean=np.nan
            CVP_duration_12=np.nan
            CVP_auc_12=np.nan
            
            NEW_MAP_preCPB_mean=np.nan #
            NEW_MAP_preCPB_CV=np.nan #
            NEW_MAP_preCPB_ARV=np.nan #
            NEW_MAP_preCPB_duration_65_=np.nan #
            NEW_MAP_preCPB_duration_100=np.nan #
            NEW_MAP_preCPB_auc_65=np.nan #
            NEW_MAP_preCPB_auc_100=np.nan #
            NEW_PP_preCPB_mean=np.nan #
            NEW_PP_preCPB_duration_60=np.nan #
            NEW_PP_preCPB_auc_60=np.nan #
            NEW_CVP_preCPB_mean=np.nan 
            NEW_CVP_preCPB_duration_12=np.nan #
            NEW_CVP_preCPB_auc_12=np.nan #

            NEW_MAP_intraCPB_mean=np.nan #
            NEW_MAP_intraCPB_CV=np.nan #
            NEW_MAP_intraCPB_ARV=np.nan #
            NEW_MAP_intraCPB_duration_65=np.nan #
            NEW_MAP_intraCPB_duration_100=np.nan #
            NEW_MAP_intraCPB_auc_65=np.nan #
            NEW_MAP_intraCPB_auc_100=np.nan #

            NEW_MAP_postCPB_mean=np.nan #
            NEW_MAP_postCPB_CV=np.nan #
            NEW_MAP_postCPB_ARV=np.nan #
            NEW_MAP_postCPB_duration_65=np.nan #
            NEW_MAP_postCPB_duration_100=np.nan #
            NEW_MAP_postCPB_auc_65=np.nan #
            NEW_MAP_postCPB_auc_100=np.nan #
            NEW_PP_postCPB_mean=np.nan #
            NEW_PP_postCPB_duration_60=np.nan #
            NEW_PP_postCPB_auc_60=np.nan #
            NEW_CVP_postCPB_mean=np.nan 
            NEW_CVP_postCPB_duration_12=np.nan #
            NEW_CVP_postCPB_auc_12=np.nan #

            NEW_PP_CPBMAPincluded_mean=np.nan #
            NEW_PP_CPBMAPincluded_duration_60=np.nan #
            NEW_PP_CPBMAPincluded_auc_60=np.nan #

            
            MAP_values_TCAexclude=[]
            MAP_values_CPBexclude=[]
            CVP_values_CPBexclude=[]
            PP_values_CPBexclude=[]
            
            MAP_values_preCPB=[]
            CVP_values_preCPB=[]
            PP_values_preCPB=[]
            
            MAP_values_intraCPB=[]
            
            MAP_values_postCPB=[]
            CVP_values_postCPB=[]
            PP_values_postCPB=[]
            
            PP_values_CPBMAPincluded=[]
            

            if df['op_type'][i]!=0:

                try:
                    anes_end_time=get_anes_end_time(i)
                    anes_start_time=get_anes_start_time(i)
                    CPB_on_time=get_CPB_on_time(i)
                    CPB_off_time=get_CPB_off_time(i)
                    CPB_end_time=get_CPB_off_time(i)

                    try:
                        TCA_start_time=get_TCA_start_time(i)
                        TCA_end_time=get_TCA_end_time(i) 
                        for j in range(len(MAP_CVP_df)):          
                            timestamp=MAP_CVP_df['datetime'][j]
                            if (timestamp>=anes_start_time and timestamp<=TCA_start_time and MAP_CVP_df['MAP_drop'][j]==0) or (timestamp>=TCA_end_time and timestamp<=anes_end_time and MAP_CVP_df['MAP_drop'][j]==0):
                                MAP_values_TCAexclude.append(MAP_CVP_df['MAP'][j].item())
                            if (timestamp>=anes_start_time and timestamp<=CPB_on_time and MAP_CVP_df['MAP_drop'][j]==0) or (timestamp>=CPB_off_time and timestamp<=anes_end_time and MAP_CVP_df['MAP_drop'][j]==0):
                                MAP_values_CPBexclude.append(MAP_CVP_df['MAP'][j].item())
                            if (timestamp>=anes_start_time and timestamp<=CPB_on_time and MAP_CVP_df['MAP_drop'][j]==0 and MAP_CVP_df['CVP_drop'][j]==0) or (timestamp>=CPB_off_time and timestamp<=anes_end_time and MAP_CVP_df['MAP_drop'][j]==0 and MAP_CVP_df['CVP_drop'][j]==0):
                                PP_values_CPBexclude.append(MAP_CVP_df['MAP'][j].item()-MAP_CVP_df['CVP'][j].item())
                            if (timestamp>=anes_start_time and timestamp<=CPB_on_time and MAP_CVP_df['MAP_drop'][j]==0):
                                MAP_values_preCPB.append(MAP_CVP_df['MAP'][j].item())
                            if (timestamp>=anes_start_time and timestamp<=CPB_on_time and MAP_CVP_df['MAP_drop'][j]==0 and MAP_CVP_df['CVP_drop'][j]==0):
                                PP_values_preCPB.append(MAP_CVP_df['MAP'][j].item()-MAP_CVP_df['CVP'][j].item())
                            if (timestamp>=CPB_on_time and timestamp<=TCA_start_time and MAP_CVP_df['MAP_drop'][j]==0) or (timestamp>=TCA_end_time and timestamp<=CPB_end_time and MAP_CVP_df['MAP_drop'][j]==0):
                                MAP_values_intraCPB.append(MAP_CVP_df['MAP'][j].item())
                            if (timestamp>=CPB_end_time and timestamp<=anes_end_time and MAP_CVP_df['MAP_drop'][j]==0):
                                MAP_values_postCPB.append(MAP_CVP_df['MAP'][j].item())
                            if (timestamp>=CPB_end_time and timestamp<=anes_end_time and MAP_CVP_df['MAP_drop'][j]==0 and MAP_CVP_df['CVP_drop'][j]==0):
                                PP_values_postCPB.append(MAP_CVP_df['MAP'][j].item()-MAP_CVP_df['CVP'][j].item())
                            if (timestamp>=anes_start_time and timestamp<=CPB_on_time and MAP_CVP_df['MAP_drop'][j]==0 and MAP_CVP_df['CVP_drop'][j]==0) or (timestamp>=CPB_off_time and timestamp<=anes_end_time and MAP_CVP_df['MAP_drop'][j]==0 and MAP_CVP_df['CVP_drop'][j]==0):
                                PP_values_CPBMAPincluded.append(MAP_CVP_df['MAP'][j].item()-MAP_CVP_df['CVP'][j].item())
                            if (timestamp>=CPB_on_time and timestamp<=TCA_start_time and MAP_CVP_df['MAP_drop'][j]==0) or (timestamp>=TCA_end_time and timestamp<=CPB_end_time and MAP_CVP_df['MAP_drop'][j]==0):
                                PP_values_CPBMAPincluded.append(MAP_CVP_df['MAP'][j].item())
                    except:
                        for j in range(len(MAP_CVP_df)):          
                            timestamp=MAP_CVP_df['datetime'][j]
                            if timestamp>=anes_start_time and timestamp<=anes_end_time and MAP_CVP_df['MAP_drop'][j]==0:
                                MAP_values_TCAexclude.append(MAP_CVP_df['MAP'][j].item())
                            if (timestamp>=anes_start_time and timestamp<=CPB_on_time and MAP_CVP_df['MAP_drop'][j]==0) or (timestamp>=CPB_off_time and timestamp<=anes_end_time and MAP_CVP_df['MAP_drop'][j]==0):
                                MAP_values_CPBexclude.append(MAP_CVP_df['MAP'][j].item())
                            if (timestamp>=anes_start_time and timestamp<=CPB_on_time and MAP_CVP_df['MAP_drop'][j]==0 and MAP_CVP_df['CVP_drop'][j]==0) or (timestamp>=CPB_off_time and timestamp<=anes_end_time and MAP_CVP_df['MAP_drop'][j]==0 and MAP_CVP_df['CVP_drop'][j]==0):
                                PP_values_CPBexclude.append(MAP_CVP_df['MAP'][j].item()-MAP_CVP_df['CVP'][j].item())
                            if (timestamp>=anes_start_time and timestamp<=CPB_on_time and MAP_CVP_df['MAP_drop'][j]==0):
                                MAP_values_preCPB.append(MAP_CVP_df['MAP'][j].item())
                            if (timestamp>=anes_start_time and timestamp<=CPB_on_time and MAP_CVP_df['MAP_drop'][j]==0 and MAP_CVP_df['CVP_drop'][j]==0):
                                PP_values_preCPB.append(MAP_CVP_df['MAP'][j].item()-MAP_CVP_df['CVP'][j].item())
                            if (timestamp>=CPB_on_time and timestamp<=CPB_end_time and MAP_CVP_df['MAP_drop'][j]==0):
                                MAP_values_intraCPB.append(MAP_CVP_df['MAP'][j].item())
                            if (timestamp>=CPB_end_time and timestamp<=anes_end_time and MAP_CVP_df['MAP_drop'][j]==0):
                                MAP_values_postCPB.append(MAP_CVP_df['MAP'][j].item())
                            if (timestamp>=CPB_end_time and timestamp<=anes_end_time and MAP_CVP_df['MAP_drop'][j]==0 and MAP_CVP_df['CVP_drop'][j]==0):
                                PP_values_postCPB.append(MAP_CVP_df['MAP'][j].item()-MAP_CVP_df['CVP'][j].item())
                            if (timestamp>=anes_start_time and timestamp<=CPB_on_time and MAP_CVP_df['MAP_drop'][j]==0 and MAP_CVP_df['CVP_drop'][j]==0) or (timestamp>=CPB_off_time and timestamp<=anes_end_time and MAP_CVP_df['MAP_drop'][j]==0 and MAP_CVP_df['CVP_drop'][j]==0):
                                PP_values_CPBMAPincluded.append(MAP_CVP_df['MAP'][j].item()-MAP_CVP_df['CVP'][j].item())
                            if (timestamp>=CPB_on_time and timestamp<=CPB_end_time and MAP_CVP_df['MAP_drop'][j]==0):
                                PP_values_CPBMAPincluded.append(MAP_CVP_df['MAP'][j].item())

                    for j in range(len(MAP_CVP_df)):          
                        timestamp=MAP_CVP_df['datetime'][j]
                        if (timestamp>=anes_start_time and timestamp<=CPB_on_time and MAP_CVP_df['CVP_drop'][j]==0) or (timestamp>=CPB_off_time and timestamp<=anes_end_time and MAP_CVP_df['CVP_drop'][j]==0):
                            CVP_values_CPBexclude.append(MAP_CVP_df['CVP'][j].item())
                        if (timestamp>=anes_start_time and timestamp<=CPB_on_time and MAP_CVP_df['CVP_drop'][j]==0):
                            CVP_values_preCPB.append(MAP_CVP_df['CVP'][j].item())
                        if (timestamp>=CPB_end_time and timestamp<=anes_end_time and MAP_CVP_df['CVP_drop'][j]==0):
                            CVP_values_postCPB.append(MAP_CVP_df['CVP'][j].item())

                    if len(MAP_values_CPBexclude)>0:
                        MAP_mean=np.nanmean(MAP_values_TCAexclude)
                        MAP_CV=np.nanstd(MAP_values_CPBexclude)/np.nanmean(MAP_values_CPBexclude)

                        real_variation=[]

                        for j in range(len(MAP_values_CPBexclude)-1):
                            real_variation.append(abs(MAP_values_CPBexclude[j+1]-MAP_values_CPBexclude[j]))

                        MAP_ARV=np.nanmean(real_variation)

                        count_65=0
                        area_65=0
                        count_100=0
                        area_100=0
                        for j in range(len(MAP_values_TCAexclude)):
                            if MAP_values_TCAexclude[j]<65:
                                count_65+=1
                                area_65+=(65-MAP_values_TCAexclude[j])
                            if MAP_values_TCAexclude[j]>100:
                                count_100+=1
                                area_100+=(MAP_values_TCAexclude[j]-100)


                        MAP_duration_65=count_65
                        MAP_duration_100=count_100
                        MAP_auc_65=area_65
                        MAP_auc_100=area_100

                    if len(PP_values_CPBexclude)>0:
                        PP_mean=np.nanmean(PP_values_CPBexclude)

                        count_60=0
                        area_60=0
                        for j in range(len(PP_values_CPBexclude)):
                            if PP_values_CPBexclude[j]<60:
                                count_60+=1
                                area_60+=(60-PP_values_CPBexclude[j])

                        PP_duration_60=count_60
                        PP_auc_60=area_60

                    if len(CVP_values_CPBexclude)>0:
                        CVP_mean=np.nanmean(CVP_values_CPBexclude)
                        count_12=0
                        area_12=0
                        for j in range(len(CVP_values_CPBexclude)):
                            if CVP_values_CPBexclude[j]>12:
                                count_12+=1
                                area_12+=(CVP_values_CPBexclude[j]-12)

                        CVP_duration_12=count_12
                        CVP_auc_12=area_12
                        
                    if len(MAP_values_preCPB)>0:
                        NEW_MAP_preCPB_mean=np.nanmean(MAP_values_preCPB)
                        NEW_MAP_preCPB_CV=np.nanstd(MAP_values_preCPB)/np.nanmean(MAP_values_preCPB)

                        real_variation=[]

                        for j in range(len(MAP_values_preCPB)-1):
                            real_variation.append(abs(MAP_values_preCPB[j+1]-MAP_values_preCPB[j]))

                        NEW_MAP_preCPB_ARV=np.nanmean(real_variation)
                        
                        count_65=0
                        area_65=0
                        count_100=0
                        area_100=0
                        for j in range(len(MAP_values_preCPB)):
                            if MAP_values_preCPB[j]<65:
                                count_65+=1
                                area_65+=(65-MAP_values_preCPB[j])
                            if MAP_values_preCPB[j]>100:
                                count_100+=1
                                area_100+=(MAP_values_preCPB[j]-100)

                        NEW_MAP_preCPB_duration_65=count_65
                        NEW_MAP_preCPB_duration_100=count_100
                        NEW_MAP_preCPB_auc_65=area_65
                        NEW_MAP_preCPB_auc_100=area_100
                        
                    if len(CVP_values_preCPB)>0:  
                        NEW_CVP_preCPB_mean=np.nanmean(CVP_values_preCPB)
                        count_12=0
                        area_12=0
                        for j in range(len(CVP_values_preCPB)):
                            if CVP_values_preCPB[j]>12:
                                count_12+=1
                                area_12+=(CVP_values_preCPB[j]-12)

                        NEW_CVP_preCPB_duration_12=count_12
                        NEW_CVP_preCPB_auc_12=area_12
                        
                    if len(PP_values_preCPB)>0:
                        NEW_PP_preCPB_mean=np.nanmean(PP_values_preCPB)

                        count_60=0
                        area_60=0
                        for j in range(len(PP_values_preCPB)):
                            if PP_values_preCPB[j]<60:
                                count_60+=1
                                area_60+=(60-PP_values_preCPB[j])

                        NEW_PP_preCPB_duration_60=count_60
                        NEW_PP_preCPB_auc_60=area_60
                        
                        
                        
                    if len(MAP_values_intraCPB)>0:
                        NEW_MAP_intraCPB_mean=np.nanmean(MAP_values_intraCPB)
                        NEW_MAP_intraCPB_CV=np.nanstd(MAP_values_intraCPB)/np.nanmean(MAP_values_intraCPB)
                        
                        real_variation=[]

                        for j in range(len(MAP_values_intraCPB)-1):
                            real_variation.append(abs(MAP_values_intraCPB[j+1]-MAP_values_intraCPB[j]))

                        NEW_MAP_intraCPB_ARV=np.nanmean(real_variation)
                        
                        
                        count_65=0
                        area_65=0
                        count_100=0
                        area_100=0
                        for j in range(len(MAP_values_intraCPB)):
                            if MAP_values_intraCPB[j]<65:
                                count_65+=1
                                area_65+=(65-MAP_values_intraCPB[j])
                            if MAP_values_intraCPB[j]>100:
                                count_100+=1
                                area_100+=(MAP_values_intraCPB[j]-100)

                        NEW_MAP_intraCPB_duration_65=count_65
                        NEW_MAP_intraCPB_duration_100=count_100
                        NEW_MAP_intraCPB_auc_65=area_65
                        NEW_MAP_intraCPB_auc_100=area_100
                        
                    if len(MAP_values_postCPB)>0:
                        NEW_MAP_postCPB_mean=np.nanmean(MAP_values_postCPB)
                        NEW_MAP_postCPB_CV=np.nanstd(MAP_values_postCPB)/np.nanmean(MAP_values_postCPB)

                        real_variation=[]

                        for j in range(len(MAP_values_postCPB)-1):
                            real_variation.append(abs(MAP_values_postCPB[j+1]-MAP_values_postCPB[j]))

                        NEW_MAP_postCPB_ARV=np.nanmean(real_variation)
                        
                        count_65=0
                        area_65=0
                        count_100=0
                        area_100=0
                        for j in range(len(MAP_values_postCPB)):
                            if MAP_values_postCPB[j]<65:
                                count_65+=1
                                area_65+=(65-MAP_values_postCPB[j])
                            if MAP_values_postCPB[j]>100:
                                count_100+=1
                                area_100+=(MAP_values_postCPB[j]-100)

                        NEW_MAP_postCPB_duration_65=count_65
                        NEW_MAP_postCPB_duration_100=count_100
                        NEW_MAP_postCPB_auc_65=area_65
                        NEW_MAP_postCPB_auc_100=area_100
                        
                    if len(CVP_values_postCPB)>0:
                        NEW_CVP_postCPB_mean=np.nanmean(CVP_values_postCPB)
                        count_12=0
                        area_12=0
                        for j in range(len(CVP_values_postCPB)):
                            if CVP_values_postCPB[j]>12:
                                count_12+=1
                                area_12+=(CVP_values_postCPB[j]-12)

                        NEW_CVP_postCPB_duration_12=count_12
                        NEW_CVP_postCPB_auc_12=area_12
                        
                    if len(PP_values_postCPB)>0:
                        NEW_PP_postCPB_mean=np.nanmean(PP_values_postCPB)

                        count_60=0
                        area_60=0
                        for j in range(len(PP_values_postCPB)):
                            if PP_values_postCPB[j]<60:
                                count_60+=1
                                area_60+=(60-PP_values_postCPB[j])

                        NEW_PP_postCPB_duration_60=count_60
                        NEW_PP_postCPB_auc_60=area_60
                        
                    if len(PP_values_CPBMAPincluded)>0:
                        NEW_PP_CPBMAPincluded_mean=np.nanmean(PP_values_CPBMAPincluded)

                        count_60=0
                        area_60=0
                        for j in range(len(PP_values_CPBMAPincluded)):
                            if PP_values_CPBMAPincluded[j]<60:
                                count_60+=1
                                area_60+=(60-PP_values_CPBMAPincluded[j])

                        NEW_PP_CPBMAPincluded_duration_60=count_60
                        NEW_PP_CPBMAPincluded_auc_60=area_60


                    MAP_mean_list.append(MAP_mean)
                    MAP_CV_list.append(MAP_CV)
                    MAP_ARV_list.append(MAP_ARV)
                    MAP_duration_65_list.append(MAP_duration_65)
                    MAP_duration_100_list.append(MAP_duration_100)
                    MAP_auc_65_list.append(MAP_auc_65)
                    MAP_auc_100_list.append(MAP_auc_100)
                    PP_mean_list.append(PP_mean)
                    PP_duration_60_list.append(PP_duration_60)
                    PP_auc_60_list.append(PP_auc_60)
                    CVP_mean_list.append(CVP_mean)
                    CVP_duration_12_list.append(CVP_duration_12)
                    CVP_auc_12_list.append(CVP_auc_12)
                    
                    NEW_MAP_preCPB_mean_list.append(NEW_MAP_preCPB_mean)
                    NEW_MAP_preCPB_CV_list.append(NEW_MAP_preCPB_CV)
                    NEW_MAP_preCPB_ARV_list.append(NEW_MAP_preCPB_ARV)
                    NEW_MAP_preCPB_duration_65_list.append(NEW_MAP_preCPB_duration_65)
                    NEW_MAP_preCPB_duration_100_list.append(NEW_MAP_preCPB_duration_100)
                    NEW_MAP_preCPB_auc_65_list.append(NEW_MAP_preCPB_auc_65)
                    NEW_MAP_preCPB_auc_100_list.append(NEW_MAP_preCPB_auc_100)
                    NEW_PP_preCPB_mean_list.append(NEW_PP_preCPB_mean)
                    NEW_PP_preCPB_duration_60_list.append(NEW_PP_preCPB_duration_60)
                    NEW_PP_preCPB_auc_60_list.append(NEW_PP_preCPB_auc_60)
                    NEW_CVP_preCPB_mean_list.append(NEW_CVP_preCPB_mean)
                    NEW_CVP_preCPB_duration_12_list.append(NEW_CVP_preCPB_duration_12)
                    NEW_CVP_preCPB_auc_12_list.append(NEW_CVP_preCPB_auc_12)

                    NEW_MAP_intraCPB_mean_list.append(NEW_MAP_intraCPB_mean)
                    NEW_MAP_intraCPB_CV_list.append(NEW_MAP_intraCPB_CV)
                    NEW_MAP_intraCPB_ARV_list.append(NEW_MAP_intraCPB_ARV)
                    NEW_MAP_intraCPB_duration_65_list.append(NEW_MAP_intraCPB_duration_65)
                    NEW_MAP_intraCPB_duration_100_list.append(NEW_MAP_intraCPB_duration_100)
                    NEW_MAP_intraCPB_auc_65_list.append(NEW_MAP_intraCPB_auc_65)
                    NEW_MAP_intraCPB_auc_100_list.append(NEW_MAP_intraCPB_auc_100)

                    NEW_MAP_postCPB_mean_list.append(NEW_MAP_postCPB_mean)
                    NEW_MAP_postCPB_CV_list.append(NEW_MAP_postCPB_CV)
                    NEW_MAP_postCPB_ARV_list.append(NEW_MAP_postCPB_ARV)
                    NEW_MAP_postCPB_duration_65_list.append(NEW_MAP_postCPB_duration_65)
                    NEW_MAP_postCPB_duration_100_list.append(NEW_MAP_postCPB_duration_100)
                    NEW_MAP_postCPB_auc_65_list.append(NEW_MAP_postCPB_auc_65)
                    NEW_MAP_postCPB_auc_100_list.append(NEW_MAP_postCPB_auc_100)
                    NEW_PP_postCPB_mean_list.append(NEW_PP_postCPB_mean)
                    NEW_PP_postCPB_duration_60_list.append(NEW_PP_postCPB_duration_60)
                    NEW_PP_postCPB_auc_60_list.append(NEW_PP_postCPB_auc_60)
                    NEW_CVP_postCPB_mean_list.append(NEW_CVP_postCPB_mean)
                    NEW_CVP_postCPB_duration_12_list.append(NEW_CVP_postCPB_duration_12)
                    NEW_CVP_postCPB_auc_12_list.append(NEW_CVP_postCPB_auc_12)

                    NEW_PP_CPBMAPincluded_mean_list.append(NEW_PP_CPBMAPincluded_mean)
                    NEW_PP_CPBMAPincluded_duration_60_list.append(NEW_PP_CPBMAPincluded_duration_60)
                    NEW_PP_CPBMAPincluded_auc_60_list.append(NEW_PP_CPBMAPincluded_auc_60)


                except:
                    print(i,pid,'error')
                    MAP_mean=np.nan
                    MAP_CV=np.nan
                    MAP_ARV=np.nan
                    MAP_duration_65=np.nan
                    MAP_duration_100=np.nan
                    MAP_auc_65=np.nan
                    MAP_auc_100=np.nan
                    PP_mean=np.nan
                    PP_duration_60=np.nan
                    PP_auc_60=np.nan
                    CVP_mean=np.nan
                    CVP_duration_12=np.nan
                    CVP_auc_12=np.nan
                    
                    NEW_MAP_preCPB_mean=np.nan #
                    NEW_MAP_preCPB_CV=np.nan #
                    NEW_MAP_preCPB_ARV=np.nan #
                    NEW_MAP_preCPB_duration_65_=np.nan #
                    NEW_MAP_preCPB_duration_100=np.nan #
                    NEW_MAP_preCPB_auc_65=np.nan #
                    NEW_MAP_preCPB_auc_100=np.nan #
                    NEW_PP_preCPB_mean=np.nan #
                    NEW_PP_preCPB_duration_60=np.nan #
                    NEW_PP_preCPB_auc_60=np.nan #
                    NEW_CVP_preCPB_mean=np.nan
                    NEW_CVP_preCPB_duration_12=np.nan #
                    NEW_CVP_preCPB_auc_12=np.nan #

                    NEW_MAP_intraCPB_mean=np.nan #
                    NEW_MAP_intraCPB_CV=np.nan #
                    NEW_MAP_intraCPB_ARV=np.nan #
                    NEW_MAP_intraCPB_duration_65=np.nan #
                    NEW_MAP_intraCPB_duration_100=np.nan #
                    NEW_MAP_intraCPB_auc_65=np.nan #
                    NEW_MAP_intraCPB_auc_100=np.nan #

                    NEW_MAP_postCPB_mean=np.nan #
                    NEW_MAP_postCPB_CV=np.nan #
                    NEW_MAP_postCPB_ARV=np.nan #
                    NEW_MAP_postCPB_duration_65=np.nan #
                    NEW_MAP_postCPB_duration_100=np.nan #
                    NEW_MAP_postCPB_auc_65=np.nan #
                    NEW_MAP_postCPB_auc_100=np.nan #
                    NEW_PP_postCPB_mean=np.nan #
                    NEW_PP_postCPB_duration_60=np.nan #
                    NEW_PP_postCPB_auc_60=np.nan #
                    NEW_CVP_postCPB_mean=np.nan
                    NEW_CVP_postCPB_duration_12=np.nan #
                    NEW_CVP_postCPB_auc_12=np.nan #

                    NEW_PP_CPBMAPincluded_mean=np.nan #
                    NEW_PP_CPBMAPincluded_duration_60=np.nan #
                    NEW_PP_CPBMAPincluded_auc_60=np.nan #

                    MAP_mean_list.append(MAP_mean)
                    MAP_CV_list.append(MAP_CV)
                    MAP_ARV_list.append(MAP_ARV)
                    MAP_duration_65_list.append(MAP_duration_65)
                    MAP_duration_100_list.append(MAP_duration_100)
                    MAP_auc_65_list.append(MAP_auc_65)
                    MAP_auc_100_list.append(MAP_auc_100)
                    PP_mean_list.append(PP_mean)
                    PP_duration_60_list.append(PP_duration_60)
                    PP_auc_60_list.append(PP_auc_60)
                    CVP_mean_list.append(CVP_mean)
                    CVP_duration_12_list.append(CVP_duration_12)
                    CVP_auc_12_list.append(CVP_auc_12)
                    
                    NEW_MAP_preCPB_mean_list.append(NEW_MAP_preCPB_mean)
                    NEW_MAP_preCPB_CV_list.append(NEW_MAP_preCPB_CV)
                    NEW_MAP_preCPB_ARV_list.append(NEW_MAP_preCPB_ARV)
                    NEW_MAP_preCPB_duration_65_list.append(NEW_MAP_preCPB_duration_65)
                    NEW_MAP_preCPB_duration_100_list.append(NEW_MAP_preCPB_duration_100)
                    NEW_MAP_preCPB_auc_65_list.append(NEW_MAP_preCPB_auc_65)
                    NEW_MAP_preCPB_auc_100_list.append(NEW_MAP_preCPB_auc_100)
                    NEW_PP_preCPB_mean_list.append(NEW_PP_preCPB_mean)
                    NEW_PP_preCPB_duration_60_list.append(NEW_PP_preCPB_duration_60)
                    NEW_PP_preCPB_auc_60_list.append(NEW_PP_preCPB_auc_60)
                    NEW_CVP_preCPB_mean_list.append(NEW_CVP_preCPB_mean)
                    NEW_CVP_preCPB_duration_12_list.append(NEW_CVP_preCPB_duration_12)
                    NEW_CVP_preCPB_auc_12_list.append(NEW_CVP_preCPB_auc_12)

                    NEW_MAP_intraCPB_mean_list.append(NEW_MAP_intraCPB_mean)
                    NEW_MAP_intraCPB_CV_list.append(NEW_MAP_intraCPB_CV)
                    NEW_MAP_intraCPB_ARV_list.append(NEW_MAP_intraCPB_ARV)
                    NEW_MAP_intraCPB_duration_65_list.append(NEW_MAP_intraCPB_duration_65)
                    NEW_MAP_intraCPB_duration_100_list.append(NEW_MAP_intraCPB_duration_100)
                    NEW_MAP_intraCPB_auc_65_list.append(NEW_MAP_intraCPB_auc_65)
                    NEW_MAP_intraCPB_auc_100_list.append(NEW_MAP_intraCPB_auc_100)

                    NEW_MAP_postCPB_mean_list.append(NEW_MAP_postCPB_mean)
                    NEW_MAP_postCPB_CV_list.append(NEW_MAP_postCPB_CV)
                    NEW_MAP_postCPB_ARV_list.append(NEW_MAP_postCPB_ARV)
                    NEW_MAP_postCPB_duration_65_list.append(NEW_MAP_postCPB_duration_65)
                    NEW_MAP_postCPB_duration_100_list.append(NEW_MAP_postCPB_duration_100)
                    NEW_MAP_postCPB_auc_65_list.append(NEW_MAP_postCPB_auc_65)
                    NEW_MAP_postCPB_auc_100_list.append(NEW_MAP_postCPB_auc_100)
                    NEW_PP_postCPB_mean_list.append(NEW_PP_postCPB_mean)
                    NEW_PP_postCPB_duration_60_list.append(NEW_PP_postCPB_duration_60)
                    NEW_PP_postCPB_auc_60_list.append(NEW_PP_postCPB_auc_60)
                    NEW_CVP_postCPB_mean_list.append(NEW_CVP_postCPB_mean)
                    NEW_CVP_postCPB_duration_12_list.append(NEW_CVP_postCPB_duration_12)
                    NEW_CVP_postCPB_auc_12_list.append(NEW_CVP_postCPB_auc_12)

                    NEW_PP_CPBMAPincluded_mean_list.append(NEW_PP_CPBMAPincluded_mean)
                    NEW_PP_CPBMAPincluded_duration_60_list.append(NEW_PP_CPBMAPincluded_duration_60)
                    NEW_PP_CPBMAPincluded_auc_60_list.append(NEW_PP_CPBMAPincluded_auc_60)
            else:
                print('wrong optype',i,pid)
                MAP_mean=np.nan
                MAP_CV=np.nan
                MAP_ARV=np.nan
                MAP_duration_65=np.nan
                MAP_duration_100=np.nan
                MAP_auc_65=np.nan
                MAP_auc_100=np.nan
                PP_mean=np.nan
                PP_duration_60=np.nan
                PP_auc_60=np.nan
                CVP_mean=np.nan
                CVP_duration_12=np.nan
                CVP_auc_12=np.nan

                NEW_MAP_preCPB_mean=np.nan #
                NEW_MAP_preCPB_CV=np.nan #
                NEW_MAP_preCPB_ARV=np.nan #
                NEW_MAP_preCPB_duration_65_=np.nan #
                NEW_MAP_preCPB_duration_100=np.nan #
                NEW_MAP_preCPB_auc_65=np.nan #
                NEW_MAP_preCPB_auc_100=np.nan #
                NEW_PP_preCPB_mean=np.nan #
                NEW_PP_preCPB_duration_60=np.nan #
                NEW_PP_preCPB_auc_60=np.nan #
                NEW_CVP_preCPB_mean=np.nan
                NEW_CVP_preCPB_duration_12=np.nan #
                NEW_CVP_preCPB_auc_12=np.nan #

                NEW_MAP_intraCPB_mean=np.nan #
                NEW_MAP_intraCPB_CV=np.nan #
                NEW_MAP_intraCPB_ARV=np.nan #
                NEW_MAP_intraCPB_duration_65=np.nan #
                NEW_MAP_intraCPB_duration_100=np.nan #
                NEW_MAP_intraCPB_auc_65=np.nan #
                NEW_MAP_intraCPB_auc_100=np.nan #

                NEW_MAP_postCPB_mean=np.nan #
                NEW_MAP_postCPB_CV=np.nan #
                NEW_MAP_postCPB_ARV=np.nan #
                NEW_MAP_postCPB_duration_65=np.nan #
                NEW_MAP_postCPB_duration_100=np.nan #
                NEW_MAP_postCPB_auc_65=np.nan #
                NEW_MAP_postCPB_auc_100=np.nan #
                NEW_PP_postCPB_mean=np.nan #
                NEW_PP_postCPB_duration_60=np.nan #
                NEW_PP_postCPB_auc_60=np.nan #
                NEW_CVP_postCPB_mean=np.nan
                NEW_CVP_postCPB_duration_12=np.nan #
                NEW_CVP_postCPB_auc_12=np.nan #

                NEW_PP_CPBMAPincluded_mean=np.nan #
                NEW_PP_CPBMAPincluded_duration_60=np.nan #
                NEW_PP_CPBMAPincluded_auc_60=np.nan #

                MAP_mean_list.append(MAP_mean)
                MAP_CV_list.append(MAP_CV)
                MAP_ARV_list.append(MAP_ARV)
                MAP_duration_65_list.append(MAP_duration_65)
                MAP_duration_100_list.append(MAP_duration_100)
                MAP_auc_65_list.append(MAP_auc_65)
                MAP_auc_100_list.append(MAP_auc_100)
                PP_mean_list.append(PP_mean)
                PP_duration_60_list.append(PP_duration_60)
                PP_auc_60_list.append(PP_auc_60)
                CVP_mean_list.append(CVP_mean)
                CVP_duration_12_list.append(CVP_duration_12)
                CVP_auc_12_list.append(CVP_auc_12)

                NEW_MAP_preCPB_mean_list.append(NEW_MAP_preCPB_mean)
                NEW_MAP_preCPB_CV_list.append(NEW_MAP_preCPB_CV)
                NEW_MAP_preCPB_ARV_list.append(NEW_MAP_preCPB_ARV)
                NEW_MAP_preCPB_duration_65_list.append(NEW_MAP_preCPB_duration_65)
                NEW_MAP_preCPB_duration_100_list.append(NEW_MAP_preCPB_duration_100)
                NEW_MAP_preCPB_auc_65_list.append(NEW_MAP_preCPB_auc_65)
                NEW_MAP_preCPB_auc_100_list.append(NEW_MAP_preCPB_auc_100)
                NEW_PP_preCPB_mean_list.append(NEW_PP_preCPB_mean)
                NEW_PP_preCPB_duration_60_list.append(NEW_PP_preCPB_duration_60)
                NEW_PP_preCPB_auc_60_list.append(NEW_PP_preCPB_auc_60)
                NEW_CVP_preCPB_mean_list.append(NEW_CVP_preCPB_mean)
                NEW_CVP_preCPB_duration_12_list.append(NEW_CVP_preCPB_duration_12)
                NEW_CVP_preCPB_auc_12_list.append(NEW_CVP_preCPB_auc_12)

                NEW_MAP_intraCPB_mean_list.append(NEW_MAP_intraCPB_mean)
                NEW_MAP_intraCPB_CV_list.append(NEW_MAP_intraCPB_CV)
                NEW_MAP_intraCPB_ARV_list.append(NEW_MAP_intraCPB_ARV)
                NEW_MAP_intraCPB_duration_65_list.append(NEW_MAP_intraCPB_duration_65)
                NEW_MAP_intraCPB_duration_100_list.append(NEW_MAP_intraCPB_duration_100)
                NEW_MAP_intraCPB_auc_65_list.append(NEW_MAP_intraCPB_auc_65)
                NEW_MAP_intraCPB_auc_100_list.append(NEW_MAP_intraCPB_auc_100)

                NEW_MAP_postCPB_mean_list.append(NEW_MAP_postCPB_mean)
                NEW_MAP_postCPB_CV_list.append(NEW_MAP_postCPB_CV)
                NEW_MAP_postCPB_ARV_list.append(NEW_MAP_postCPB_ARV)
                NEW_MAP_postCPB_duration_65_list.append(NEW_MAP_postCPB_duration_65)
                NEW_MAP_postCPB_duration_100_list.append(NEW_MAP_postCPB_duration_100)
                NEW_MAP_postCPB_auc_65_list.append(NEW_MAP_postCPB_auc_65)
                NEW_MAP_postCPB_auc_100_list.append(NEW_MAP_postCPB_auc_100)
                NEW_PP_postCPB_mean_list.append(NEW_PP_postCPB_mean)
                NEW_PP_postCPB_duration_60_list.append(NEW_PP_postCPB_duration_60)
                NEW_PP_postCPB_auc_60_list.append(NEW_PP_postCPB_auc_60)
                NEW_CVP_postCPB_mean_list.append(NEW_CVP_postCPB_mean)
                NEW_CVP_postCPB_duration_12_list.append(NEW_CVP_postCPB_duration_12)
                NEW_CVP_postCPB_auc_12_list.append(NEW_CVP_postCPB_auc_12)

                NEW_PP_CPBMAPincluded_mean_list.append(NEW_PP_CPBMAPincluded_mean)
                NEW_PP_CPBMAPincluded_duration_60_list.append(NEW_PP_CPBMAPincluded_duration_60)
                NEW_PP_CPBMAPincluded_auc_60_list.append(NEW_PP_CPBMAPincluded_auc_60)
                
        else:
            print(i,pid,'====excluded====')

            MAP_mean=np.nan
            MAP_CV=np.nan
            MAP_ARV=np.nan
            MAP_duration_65=np.nan
            MAP_duration_100=np.nan
            MAP_auc_65=np.nan
            MAP_auc_100=np.nan
            PP_mean=np.nan
            PP_duration_60=np.nan
            PP_auc_60=np.nan
            CVP_mean=np.nan
            CVP_duration_12=np.nan
            CVP_auc_12=np.nan

            NEW_MAP_preCPB_mean=np.nan #
            NEW_MAP_preCPB_CV=np.nan #
            NEW_MAP_preCPB_ARV=np.nan #
            NEW_MAP_preCPB_duration_65_=np.nan #
            NEW_MAP_preCPB_duration_100=np.nan #
            NEW_MAP_preCPB_auc_65=np.nan #
            NEW_MAP_preCPB_auc_100=np.nan #
            NEW_PP_preCPB_mean=np.nan #
            NEW_PP_preCPB_duration_60=np.nan #
            NEW_PP_preCPB_auc_60=np.nan #
            NEW_CVP_preCPB_mean=np.nan
            NEW_CVP_preCPB_duration_12=np.nan #
            NEW_CVP_preCPB_auc_12=np.nan #

            NEW_MAP_intraCPB_mean=np.nan #
            NEW_MAP_intraCPB_CV=np.nan #
            NEW_MAP_intraCPB_ARV=np.nan #
            NEW_MAP_intraCPB_duration_65=np.nan #
            NEW_MAP_intraCPB_duration_100=np.nan #
            NEW_MAP_intraCPB_auc_65=np.nan #
            NEW_MAP_intraCPB_auc_100=np.nan #

            NEW_MAP_postCPB_mean=np.nan #
            NEW_MAP_postCPB_CV=np.nan #
            NEW_MAP_postCPB_ARV=np.nan #
            NEW_MAP_postCPB_duration_65=np.nan #
            NEW_MAP_postCPB_duration_100=np.nan #
            NEW_MAP_postCPB_auc_65=np.nan #
            NEW_MAP_postCPB_auc_100=np.nan #
            NEW_PP_postCPB_mean=np.nan #
            NEW_PP_postCPB_duration_60=np.nan #
            NEW_PP_postCPB_auc_60=np.nan #
            NEW_CVP_postCPB_mean=np.nan
            NEW_CVP_postCPB_duration_12=np.nan #
            NEW_CVP_postCPB_auc_12=np.nan #

            NEW_PP_CPBMAPincluded_mean=np.nan #
            NEW_PP_CPBMAPincluded_duration_60=np.nan #
            NEW_PP_CPBMAPincluded_auc_60=np.nan #

            MAP_mean_list.append(MAP_mean)
            MAP_CV_list.append(MAP_CV)
            MAP_ARV_list.append(MAP_ARV)
            MAP_duration_65_list.append(MAP_duration_65)
            MAP_duration_100_list.append(MAP_duration_100)
            MAP_auc_65_list.append(MAP_auc_65)
            MAP_auc_100_list.append(MAP_auc_100)
            PP_mean_list.append(PP_mean)
            PP_duration_60_list.append(PP_duration_60)
            PP_auc_60_list.append(PP_auc_60)
            CVP_mean_list.append(CVP_mean)
            CVP_duration_12_list.append(CVP_duration_12)
            CVP_auc_12_list.append(CVP_auc_12)

            NEW_MAP_preCPB_mean_list.append(NEW_MAP_preCPB_mean)
            NEW_MAP_preCPB_CV_list.append(NEW_MAP_preCPB_CV)
            NEW_MAP_preCPB_ARV_list.append(NEW_MAP_preCPB_ARV)
            NEW_MAP_preCPB_duration_65_list.append(NEW_MAP_preCPB_duration_65)
            NEW_MAP_preCPB_duration_100_list.append(NEW_MAP_preCPB_duration_100)
            NEW_MAP_preCPB_auc_65_list.append(NEW_MAP_preCPB_auc_65)
            NEW_MAP_preCPB_auc_100_list.append(NEW_MAP_preCPB_auc_100)
            NEW_PP_preCPB_mean_list.append(NEW_PP_preCPB_mean)
            NEW_PP_preCPB_duration_60_list.append(NEW_PP_preCPB_duration_60)
            NEW_PP_preCPB_auc_60_list.append(NEW_PP_preCPB_auc_60)
            NEW_CVP_preCPB_mean_list.append(NEW_CVP_preCPB_mean)
            NEW_CVP_preCPB_duration_12_list.append(NEW_CVP_preCPB_duration_12)
            NEW_CVP_preCPB_auc_12_list.append(NEW_CVP_preCPB_auc_12)

            NEW_MAP_intraCPB_mean_list.append(NEW_MAP_intraCPB_mean)
            NEW_MAP_intraCPB_CV_list.append(NEW_MAP_intraCPB_CV)
            NEW_MAP_intraCPB_ARV_list.append(NEW_MAP_intraCPB_ARV)
            NEW_MAP_intraCPB_duration_65_list.append(NEW_MAP_intraCPB_duration_65)
            NEW_MAP_intraCPB_duration_100_list.append(NEW_MAP_intraCPB_duration_100)
            NEW_MAP_intraCPB_auc_65_list.append(NEW_MAP_intraCPB_auc_65)
            NEW_MAP_intraCPB_auc_100_list.append(NEW_MAP_intraCPB_auc_100)

            NEW_MAP_postCPB_mean_list.append(NEW_MAP_postCPB_mean)
            NEW_MAP_postCPB_CV_list.append(NEW_MAP_postCPB_CV)
            NEW_MAP_postCPB_ARV_list.append(NEW_MAP_postCPB_ARV)
            NEW_MAP_postCPB_duration_65_list.append(NEW_MAP_postCPB_duration_65)
            NEW_MAP_postCPB_duration_100_list.append(NEW_MAP_postCPB_duration_100)
            NEW_MAP_postCPB_auc_65_list.append(NEW_MAP_postCPB_auc_65)
            NEW_MAP_postCPB_auc_100_list.append(NEW_MAP_postCPB_auc_100)
            NEW_PP_postCPB_mean_list.append(NEW_PP_postCPB_mean)
            NEW_PP_postCPB_duration_60_list.append(NEW_PP_postCPB_duration_60)
            NEW_PP_postCPB_auc_60_list.append(NEW_PP_postCPB_auc_60)
            NEW_CVP_postCPB_mean_list.append(NEW_CVP_postCPB_mean)
            NEW_CVP_postCPB_duration_12_list.append(NEW_CVP_postCPB_duration_12)
            NEW_CVP_postCPB_auc_12_list.append(NEW_CVP_postCPB_auc_12)

            NEW_PP_CPBMAPincluded_mean_list.append(NEW_PP_CPBMAPincluded_mean)
            NEW_PP_CPBMAPincluded_duration_60_list.append(NEW_PP_CPBMAPincluded_duration_60)
            NEW_PP_CPBMAPincluded_auc_60_list.append(NEW_PP_CPBMAPincluded_auc_60)

    except:
        print('====nofiles pid',pid)
        MAP_mean=np.nan
        MAP_CV=np.nan
        MAP_ARV=np.nan
        MAP_duration_65=np.nan
        MAP_duration_100=np.nan
        MAP_auc_65=np.nan
        MAP_auc_100=np.nan
        PP_mean=np.nan
        PP_duration_60=np.nan
        PP_auc_60=np.nan
        CVP_mean=np.nan
        CVP_duration_12=np.nan
        CVP_auc_12=np.nan

        NEW_MAP_preCPB_mean=np.nan #
        NEW_MAP_preCPB_CV=np.nan #
        NEW_MAP_preCPB_ARV=np.nan #
        NEW_MAP_preCPB_duration_65_=np.nan #
        NEW_MAP_preCPB_duration_100=np.nan #
        NEW_MAP_preCPB_auc_65=np.nan #
        NEW_MAP_preCPB_auc_100=np.nan #
        NEW_PP_preCPB_mean=np.nan #
        NEW_PP_preCPB_duration_60=np.nan #
        NEW_PP_preCPB_auc_60=np.nan #
        NEW_CVP_preCPB_mean=np.nan
        NEW_CVP_preCPB_duration_12=np.nan #
        NEW_CVP_preCPB_auc_12=np.nan #

        NEW_MAP_intraCPB_mean=np.nan #
        NEW_MAP_intraCPB_CV=np.nan #
        NEW_MAP_intraCPB_ARV=np.nan #
        NEW_MAP_intraCPB_duration_65=np.nan #
        NEW_MAP_intraCPB_duration_100=np.nan #
        NEW_MAP_intraCPB_auc_65=np.nan #
        NEW_MAP_intraCPB_auc_100=np.nan #

        NEW_MAP_postCPB_mean=np.nan #
        NEW_MAP_postCPB_CV=np.nan #
        NEW_MAP_postCPB_ARV=np.nan #
        NEW_MAP_postCPB_duration_65=np.nan #
        NEW_MAP_postCPB_duration_100=np.nan #
        NEW_MAP_postCPB_auc_65=np.nan #
        NEW_MAP_postCPB_auc_100=np.nan #
        NEW_PP_postCPB_mean=np.nan #
        NEW_PP_postCPB_duration_60=np.nan #
        NEW_PP_postCPB_auc_60=np.nan #
        NEW_CVP_postCPB_mean=np.nan
        NEW_CVP_postCPB_duration_12=np.nan #
        NEW_CVP_postCPB_auc_12=np.nan #

        NEW_PP_CPBMAPincluded_mean=np.nan #
        NEW_PP_CPBMAPincluded_duration_60=np.nan #
        NEW_PP_CPBMAPincluded_auc_60=np.nan #

        MAP_mean_list.append(MAP_mean)
        MAP_CV_list.append(MAP_CV)
        MAP_ARV_list.append(MAP_ARV)
        MAP_duration_65_list.append(MAP_duration_65)
        MAP_duration_100_list.append(MAP_duration_100)
        MAP_auc_65_list.append(MAP_auc_65)
        MAP_auc_100_list.append(MAP_auc_100)
        PP_mean_list.append(PP_mean)
        PP_duration_60_list.append(PP_duration_60)
        PP_auc_60_list.append(PP_auc_60)
        CVP_mean_list.append(CVP_mean)
        CVP_duration_12_list.append(CVP_duration_12)
        CVP_auc_12_list.append(CVP_auc_12)

        NEW_MAP_preCPB_mean_list.append(NEW_MAP_preCPB_mean)
        NEW_MAP_preCPB_CV_list.append(NEW_MAP_preCPB_CV)
        NEW_MAP_preCPB_ARV_list.append(NEW_MAP_preCPB_ARV)
        NEW_MAP_preCPB_duration_65_list.append(NEW_MAP_preCPB_duration_65)
        NEW_MAP_preCPB_duration_100_list.append(NEW_MAP_preCPB_duration_100)
        NEW_MAP_preCPB_auc_65_list.append(NEW_MAP_preCPB_auc_65)
        NEW_MAP_preCPB_auc_100_list.append(NEW_MAP_preCPB_auc_100)
        NEW_PP_preCPB_mean_list.append(NEW_PP_preCPB_mean)
        NEW_PP_preCPB_duration_60_list.append(NEW_PP_preCPB_duration_60)
        NEW_PP_preCPB_auc_60_list.append(NEW_PP_preCPB_auc_60)
        NEW_CVP_preCPB_mean_list.append(NEW_CVP_preCPB_mean)
        NEW_CVP_preCPB_duration_12_list.append(NEW_CVP_preCPB_duration_12)
        NEW_CVP_preCPB_auc_12_list.append(NEW_CVP_preCPB_auc_12)

        NEW_MAP_intraCPB_mean_list.append(NEW_MAP_intraCPB_mean)
        NEW_MAP_intraCPB_CV_list.append(NEW_MAP_intraCPB_CV)
        NEW_MAP_intraCPB_ARV_list.append(NEW_MAP_intraCPB_ARV)
        NEW_MAP_intraCPB_duration_65_list.append(NEW_MAP_intraCPB_duration_65)
        NEW_MAP_intraCPB_duration_100_list.append(NEW_MAP_intraCPB_duration_100)
        NEW_MAP_intraCPB_auc_65_list.append(NEW_MAP_intraCPB_auc_65)
        NEW_MAP_intraCPB_auc_100_list.append(NEW_MAP_intraCPB_auc_100)

        NEW_MAP_postCPB_mean_list.append(NEW_MAP_postCPB_mean)
        NEW_MAP_postCPB_CV_list.append(NEW_MAP_postCPB_CV)
        NEW_MAP_postCPB_ARV_list.append(NEW_MAP_postCPB_ARV)
        NEW_MAP_postCPB_duration_65_list.append(NEW_MAP_postCPB_duration_65)
        NEW_MAP_postCPB_duration_100_list.append(NEW_MAP_postCPB_duration_100)
        NEW_MAP_postCPB_auc_65_list.append(NEW_MAP_postCPB_auc_65)
        NEW_MAP_postCPB_auc_100_list.append(NEW_MAP_postCPB_auc_100)
        NEW_PP_postCPB_mean_list.append(NEW_PP_postCPB_mean)
        NEW_PP_postCPB_duration_60_list.append(NEW_PP_postCPB_duration_60)
        NEW_PP_postCPB_auc_60_list.append(NEW_PP_postCPB_auc_60)
        NEW_CVP_postCPB_mean_list.append(NEW_CVP_postCPB_mean)
        NEW_CVP_postCPB_duration_12_list.append(NEW_CVP_postCPB_duration_12)
        NEW_CVP_postCPB_auc_12_list.append(NEW_CVP_postCPB_auc_12)

        NEW_PP_CPBMAPincluded_mean_list.append(NEW_PP_CPBMAPincluded_mean)
        NEW_PP_CPBMAPincluded_duration_60_list.append(NEW_PP_CPBMAPincluded_duration_60)
        NEW_PP_CPBMAPincluded_auc_60_list.append(NEW_PP_CPBMAPincluded_auc_60)
        
        
        
    if i%100==0:
        print(i)
        
        


In [ ]:
df['MAP_mean_list']=MAP_mean_list
df['MAP_CV_list']=MAP_CV_list
df['MAP_ARV_list']=MAP_ARV_list
df['MAP_duration_65_list']=MAP_duration_65_list
df['MAP_duration_100_list']=MAP_duration_100_list
df['MAP_auc_65_list']=MAP_auc_65_list
df['MAP_auc_100_list']=MAP_auc_100_list
df['PP_mean_list']=PP_mean_list
df['PP_duration_60_list']=PP_duration_60_list
df['PP_auc_60_list']=PP_auc_60_list
df['CVP_mean_list']=CVP_mean_list
df['CVP_duration_12_list']=CVP_duration_12_list
df['CVP_auc_12_list']=CVP_auc_12_list

df['NEW_MAP_preCPB_mean_list']=NEW_MAP_preCPB_mean_list
df['NEW_MAP_preCPB_CV_list']=NEW_MAP_preCPB_CV_list
df['NEW_MAP_preCPB_ARV_list']=NEW_MAP_preCPB_ARV_list
df['NEW_MAP_preCPB_duration_65_list']=NEW_MAP_preCPB_duration_65_list
df['NEW_MAP_preCPB_duration_100_list']=NEW_MAP_preCPB_duration_100_list
df['NEW_MAP_preCPB_auc_65_list']=NEW_MAP_preCPB_auc_65_list
df['NEW_MAP_preCPB_auc_100_list']=NEW_MAP_preCPB_auc_100_list
df['NEW_PP_preCPB_mean_list']=NEW_PP_preCPB_mean_list
df['NEW_PP_preCPB_duration_60_list']=NEW_PP_preCPB_duration_60_list
df['NEW_PP_preCPB_auc_60_list']=NEW_PP_preCPB_auc_60_list
df['NEW_CVP_preCPB_mean_list']=NEW_CVP_preCPB_mean_list
df['NEW_CVP_preCPB_duration_12_list']=NEW_CVP_preCPB_duration_12_list
df['NEW_CVP_preCPB_auc_12_list']=NEW_CVP_preCPB_auc_12_list

df['NEW_MAP_intraCPB_mean_list']=NEW_MAP_intraCPB_mean_list
df['NEW_MAP_intraCPB_CV_list']=NEW_MAP_intraCPB_CV_list
df['NEW_MAP_intraCPB_ARV_list']=NEW_MAP_intraCPB_ARV_list
df['NEW_MAP_intraCPB_duration_65_list']=NEW_MAP_intraCPB_duration_65_list
df['NEW_MAP_intraCPB_duration_100_list']=NEW_MAP_intraCPB_duration_100_list
df['NEW_MAP_intraCPB_auc_65_list']=NEW_MAP_intraCPB_auc_65_list
df['NEW_MAP_intraCPB_auc_100_list']=NEW_MAP_intraCPB_auc_100_list

df['NEW_MAP_postCPB_mean_list']=NEW_MAP_postCPB_mean_list
df['NEW_MAP_postCPB_CV_list']=NEW_MAP_postCPB_CV_list
df['NEW_MAP_postCPB_ARV_list']=NEW_MAP_postCPB_ARV_list
df['NEW_MAP_postCPB_duration_65_list']=NEW_MAP_postCPB_duration_65_list
df['NEW_MAP_postCPB_duration_100_list']=NEW_MAP_postCPB_duration_100_list
df['NEW_MAP_postCPB_auc_65_list']=NEW_MAP_postCPB_auc_65_list
df['NEW_MAP_postCPB_auc_100_list']=NEW_MAP_postCPB_auc_100_list
df['NEW_PP_postCPB_mean_list']=NEW_PP_postCPB_mean_list
df['NEW_PP_postCPB_duration_60_list']=NEW_PP_postCPB_duration_60_list
df['NEW_PP_postCPB_auc_60_list']=NEW_PP_postCPB_auc_60_list
df['NEW_CVP_postCPB_mean_list']=NEW_CVP_postCPB_mean_list
df['NEW_CVP_postCPB_duration_12_list']=NEW_CVP_postCPB_duration_12_list
df['NEW_CVP_postCPB_auc_12_list']=NEW_CVP_postCPB_auc_12_list

df['NEW_PP_CPBMAPincluded_mean_list']=NEW_PP_CPBMAPincluded_mean_list
df['NEW_PP_CPBMAPincluded_duration_60_list']=NEW_PP_CPBMAPincluded_duration_60_list
df['NEW_PP_CPBMAPincluded_auc_60_list']=NEW_PP_CPBMAPincluded_auc_60_list

In [ ]:
df_temp=df[['NO','NEW_MAP_intraCPB_CV_list','NEW_MAP_intraCPB_ARV_list']]

In [ ]:
df_temp.to_csv('20240820_NEW_MAP_CV_ARV.csv',index=False)

In [ ]:
plt.hist(df['NEW_MAP_intraCPB_CV_list'])

In [ ]:
plt.hist(df['NEW_CVP_postCPB_mean_list'])

In [ ]:
plt.hist(df['NEW_CVP_postCPB_mean_list'])

In [ ]:
plt.hist(df['NEW_PP_CPBMAPincluded_auc_60_list'])

In [ ]:
plt.hist(df['NEW_MAP_postCPB_mean_list'])

In [ ]:
plt.hist(df['MAP_auc_65_list'])

In [ ]:
plt.hist(df['NEW_MAP_preCPB_auc_100_list'])

In [ ]:
for i in range(2050,len(df)):
    pid=df['NO'][i]
    CI_total=pd.DataFrame(columns=['Time','CI'])
    for filefile in ['파일이름_01','파일이름_02','파일이름_03','파일이름_04','파일이름_05','파일이름_06','파일이름_07','파일이름_08']:      
        try:
            dat=pd.read_csv(findfile(df[filefile][i]))
            try:
                CI_columns=['Time']
                CI_columns2=[col for col in dat.columns if '/CI' in col and '/CI_stat' not in col]
                CI_columns=CI_columns+CI_columns2
                CI=dat[CI_columns]
                
                CI=CI.dropna(axis=0,subset=CI_columns[1:],how='all')
                CI.reset_index(inplace=True,drop=True)
                CI['CI_final'] = CI.apply(find_first_non_null, axis=1)
                CI_temp=CI[['Time','CI_final']]
                CI_temp.columns=['Time','CI']

                CI_total=pd.concat([CI_total,CI_temp])
            except:
                pass   
        except:
            pass


    CI_total=CI_total.dropna(axis=0)

    CI_total.sort_values(by=['Time'],axis=0,inplace=True)

    CI_total.reset_index(inplace=True,drop=True)

    CI_total['year']=CI_total['Time'].str[:4]
    CI_total['month']=CI_total['Time'].str[5:7]
    CI_total['day']=CI_total['Time'].str[8:10]
    CI_total['hour']=CI_total['Time'].str[11:13]
    CI_total['minute']=CI_total['Time'].str[14:16]
    CI_total['second']=CI_total['Time'].str[17:19]
    CI_total['year']=pd.to_numeric(CI_total['year'])
    CI_total['month']=pd.to_numeric(CI_total['month'])
    CI_total['day']=pd.to_numeric(CI_total['day'])
    CI_total['hour']=pd.to_numeric(CI_total['hour'])
    CI_total['minute']=pd.to_numeric(CI_total['minute'])
    CI_total['second']=pd.to_numeric(CI_total['second'])
    CI_total['second2']=(CI_total['second']//5)*5

    CI_total['timetime']=CI_total['Time'].str[:17]+('0'+CI_total['second2'].astype(str)).str[-2:]
    CI_total['datetime']=pd.to_datetime(CI_total['timetime'],format="%Y-%m-%d %H:%M:%S")

    CI_total2=CI_total[['datetime','CI']]

    CI_total3=CI_total2[CI_total2['CI']!=0]


    CI_total3.drop_duplicates(inplace=True,subset=['datetime'],keep='first')
    CI_total3.reset_index(inplace=True,drop=True)

    CI_total3.to_csv(savepath+"CI/"+str(pid)+'.csv')


In [ ]:
CI_avail=[]

    
for i in range(len(df)):
    pid=df['NO'][i]
    try:
        if True: 
            if df['op_type'][i]!=0:
                try:
                    anes_end_time=get_anes_end_time(i)
                    anes_start_time=get_anes_start_time(i)
                    CPB_on_time=get_CPB_on_time(i)
                    CPB_off_time=get_CPB_off_time(i)

                    total_anes_minus_CPB_time=(anes_end_time-anes_start_time).seconds-(CPB_off_time-CPB_on_time).seconds

                    CI_df=pd.read_csv(savepath+"CI/"+str(pid)+'.csv')
                    CI_df['datetime']=pd.to_datetime(CI_df['datetime'],format="%Y-%m-%d %H:%M:%S")
                    count=0

                    for j in range(len(CI_df)):

                        timestamp=CI_df['datetime'][j]
                        if (timestamp>=anes_start_time and timestamp<=CPB_on_time) or (timestamp>=CPB_off_time and timestamp<=anes_end_time):
                            count+=1

                    CI_avail.append(count/(total_anes_minus_CPB_time//5))          

                except:
                    print(i,'error')  
                    CI_avail.append(0)
            else:
                print(i,pid,'optypeerror')
                CI_avail.append(0)
        else:
            print(i,'===excluded===')
            CI_avail.append(0)
    except:
        print(i,pid,'nofiles')
        CI_avail.append(0)

df['CI_avail']=CI_avail

In [ ]:
plt.hist(CI_avail)

In [ ]:
CI_mean_list=[]
CI_duration_2_list=[]
CI_auc_2_list=[]

NEW_CI_preCPB_mean_list=[]
NEW_CI_preCPB_duration_2_list=[]
NEW_CI_preCPB_auc_2_list=[]

NEW_CI_postCPB_mean_list=[]
NEW_CI_postCPB_duration_2_list=[]
NEW_CI_postCPB_auc_2_list=[]

for i in range(len(df)):
    pid=df['NO'][i]
    try:
        if True: 
            CI_df=pd.read_csv(savepath+"CI/"+str(pid)+'.csv')
            CI_df['datetime']=pd.to_datetime(CI_df['datetime'],format="%Y-%m-%d %H:%M:%S")

            CI_mean=np.nan
            CI_duration_2=np.nan
            CI_auc_2=np.nan
            
            NEW_CI_preCPB_mean=np.nan
            NEW_CI_preCPB_duration_2=np.nan
            NEW_CI_preCPB_auc_2=np.nan

            NEW_CI_postCPB_mean=np.nan
            NEW_CI_postCPB_duration_2=np.nan
            NEW_CI_postCPB_auc_2=np.nan

            CI_values=[]
            CI_preCPB_values=[]
            CI_postCPB_values=[]

            if df['op_type'][i]!=0:

                try:
                    anes_end_time=get_anes_end_time(i)
                    anes_start_time=get_anes_start_time(i)
                    CPB_on_time=get_CPB_on_time(i)
                    CPB_off_time=get_CPB_off_time(i)

                    for j in range(len(CI_df)):          
                        timestamp=CI_df['datetime'][j]
                        if (timestamp>=anes_start_time and timestamp<=CPB_on_time) or (timestamp>=CPB_off_time and timestamp<=anes_end_time):
                            CI_values.append(CI_df['CI'][j].item())
                        if (timestamp>=anes_start_time and timestamp<=CPB_on_time):
                            CI_preCPB_values.append(CI_df['CI'][j].item())
                        if (timestamp>=CPB_off_time and timestamp<=anes_end_time):
                            CI_postCPB_values.append(CI_df['CI'][j].item())

                    if len(CI_values)>0:
                        CI_mean=np.nanmean(CI_values)
                        count_2=0
                        area_2=0

                        for j in range(len(CI_values)):
                            if CI_values[j]<2:
                                count_2+=1
                                area_2+=(2-CI_values[j])


                        CI_duration_2=count_2
                        CI_auc_2=area_2
                        
                    if len(CI_preCPB_values)>0:
                        NEW_CI_preCPB_mean=np.nanmean(CI_preCPB_values)
                        count_2=0
                        area_2=0

                        for j in range(len(CI_preCPB_values)):
                            if CI_preCPB_values[j]<2:
                                count_2+=1
                                area_2+=(2-CI_preCPB_values[j])


                        NEW_CI_preCPB_duration_2=count_2
                        NEW_CI_preCPB_auc_2=area_2
                        
                    if len(CI_postCPB_values)>0:
                        NEW_CI_postCPB_mean=np.nanmean(CI_postCPB_values)
                        count_2=0
                        area_2=0

                        for j in range(len(CI_postCPB_values)):
                            if CI_postCPB_values[j]<2:
                                count_2+=1
                                area_2+=(2-CI_postCPB_values[j])


                        NEW_CI_postCPB_duration_2=count_2
                        NEW_CI_postCPB_auc_2=area_2


                    CI_mean_list.append(CI_mean)
                    CI_duration_2_list.append(CI_duration_2)
                    CI_auc_2_list.append(CI_auc_2)
                    
                    NEW_CI_preCPB_mean_list.append(NEW_CI_preCPB_mean)
                    NEW_CI_preCPB_duration_2_list.append(NEW_CI_preCPB_duration_2)
                    NEW_CI_preCPB_auc_2_list.append(NEW_CI_preCPB_auc_2)

                    NEW_CI_postCPB_mean_list.append(NEW_CI_postCPB_mean)
                    NEW_CI_postCPB_duration_2_list.append(NEW_CI_postCPB_duration_2)
                    NEW_CI_postCPB_auc_2_list.append(NEW_CI_postCPB_auc_2)
                    #print(i)


                except:
                    print(i,'error')
                    CI_mean=np.nan
                    CI_duration_2=np.nan
                    CI_auc_2=np.nan
                    
                    NEW_CI_preCPB_mean=np.nan
                    NEW_CI_preCPB_duration_2=np.nan
                    NEW_CI_preCPB_auc_2=np.nan
                    
                    NEW_CI_postCPB_mean=np.nan
                    NEW_CI_postCPB_duration_2=np.nan
                    NEW_CI_postCPB_auc_2=np.nan

                    CI_mean_list.append(CI_mean)
                    CI_duration_2_list.append(CI_duration_2)
                    CI_auc_2_list.append(CI_auc_2)
                    
                    NEW_CI_preCPB_mean_list.append(NEW_CI_preCPB_mean)
                    NEW_CI_preCPB_duration_2_list.append(NEW_CI_preCPB_duration_2)
                    NEW_CI_preCPB_auc_2_list.append(NEW_CI_preCPB_auc_2)

                    NEW_CI_postCPB_mean_list.append(NEW_CI_postCPB_mean)
                    NEW_CI_postCPB_duration_2_list.append(NEW_CI_postCPB_duration_2)
                    NEW_CI_postCPB_auc_2_list.append(NEW_CI_postCPB_auc_2)
                    
            else:
                print(i,pid,'optypeerror')
                CI_mean=np.nan
                CI_duration_2=np.nan
                CI_auc_2=np.nan

                NEW_CI_preCPB_mean=np.nan
                NEW_CI_preCPB_duration_2=np.nan
                NEW_CI_preCPB_auc_2=np.nan

                NEW_CI_postCPB_mean=np.nan
                NEW_CI_postCPB_duration_2=np.nan
                NEW_CI_postCPB_auc_2=np.nan

                CI_mean_list.append(CI_mean)
                CI_duration_2_list.append(CI_duration_2)
                CI_auc_2_list.append(CI_auc_2)

                NEW_CI_preCPB_mean_list.append(NEW_CI_preCPB_mean)
                NEW_CI_preCPB_duration_2_list.append(NEW_CI_preCPB_duration_2)
                NEW_CI_preCPB_auc_2_list.append(NEW_CI_preCPB_auc_2)

                NEW_CI_postCPB_mean_list.append(NEW_CI_postCPB_mean)
                NEW_CI_postCPB_duration_2_list.append(NEW_CI_postCPB_duration_2)
                NEW_CI_postCPB_auc_2_list.append(NEW_CI_postCPB_auc_2)
        else:
            print(i,'===excluded===')

            CI_meanCI_mean=np.nan
            CI_duration_2=np.nan
            CI_auc_2=np.nan

            NEW_CI_preCPB_mean=np.nan
            NEW_CI_preCPB_duration_2=np.nan
            NEW_CI_preCPB_auc_2=np.nan

            NEW_CI_postCPB_mean=np.nan
            NEW_CI_postCPB_duration_2=np.nan
            NEW_CI_postCPB_auc_2=np.nan

            CI_mean_list.append(CI_mean)
            CI_duration_2_list.append(CI_duration_2)
            CI_auc_2_list.append(CI_auc_2)

            NEW_CI_preCPB_mean_list.append(NEW_CI_preCPB_mean)
            NEW_CI_preCPB_duration_2_list.append(NEW_CI_preCPB_duration_2)
            NEW_CI_preCPB_auc_2_list.append(NEW_CI_preCPB_auc_2)

            NEW_CI_postCPB_mean_list.append(NEW_CI_postCPB_mean)
            NEW_CI_postCPB_duration_2_list.append(NEW_CI_postCPB_duration_2)
            NEW_CI_postCPB_auc_2_list.append(NEW_CI_postCPB_auc_2)
    except:
        print(i,pid,'nofile')
        CI_meanCI_mean=np.nan
        CI_duration_2=np.nan
        CI_auc_2=np.nan

        NEW_CI_preCPB_mean=np.nan
        NEW_CI_preCPB_duration_2=np.nan
        NEW_CI_preCPB_auc_2=np.nan

        NEW_CI_postCPB_mean=np.nan
        NEW_CI_postCPB_duration_2=np.nan
        NEW_CI_postCPB_auc_2=np.nan

        CI_mean_list.append(CI_mean)
        CI_duration_2_list.append(CI_duration_2)
        CI_auc_2_list.append(CI_auc_2)

        NEW_CI_preCPB_mean_list.append(NEW_CI_preCPB_mean)
        NEW_CI_preCPB_duration_2_list.append(NEW_CI_preCPB_duration_2)
        NEW_CI_preCPB_auc_2_list.append(NEW_CI_preCPB_auc_2)

        NEW_CI_postCPB_mean_list.append(NEW_CI_postCPB_mean)
        NEW_CI_postCPB_duration_2_list.append(NEW_CI_postCPB_duration_2)
        NEW_CI_postCPB_auc_2_list.append(NEW_CI_postCPB_auc_2)
        
df['CI_mean_list']=CI_mean_list
df['CI_duration_2_list']=CI_duration_2_list
df['CI_auc_2_list']=CI_auc_2_list

df['NEW_CI_preCPB_mean_list']=NEW_CI_preCPB_mean_list
df['NEW_CI_preCPB_duration_2_list']=NEW_CI_preCPB_duration_2_list
df['NEW_CI_preCPB_auc_2_list']=NEW_CI_preCPB_auc_2_list

df['NEW_CI_postCPB_mean_list']=NEW_CI_postCPB_mean_list
df['NEW_CI_postCPB_duration_2_list']=NEW_CI_postCPB_duration_2_list
df['NEW_CI_postCPB_auc_2_list']=NEW_CI_postCPB_auc_2_list

In [ ]:
plt.hist(df['CI_duration_2_list'])

In [ ]:
plt.hist(df['NEW_CI_postCPB_mean_list'])

In [ ]:
df.to_csv('20240625_AKI_final2.csv',index=False)

In [ ]:
df